In [1]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

In [2]:
path = 'Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/air_1587/'

In [3]:
from dsgtools import azure
## link to 20 ipynb
base = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/air_1587/1108_air_1587_business_shell_v31_base_w20231107-084618_sas_layout_busshell.csv", 
                    reader = pd.read_csv, encoding='iso-8859-1', verbose = False, low_memory = False)
test = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/air_1587/1108_air_1587_business_shell_v31_test_w20231107-113955_sas_layout_busshell.csv", 
                    reader = pd.read_csv, encoding='iso-8859-1', verbose = False, low_memory = False)
print(base.shape)
print(test.shape)

(100000, 3115)
(99716, 3115)


In [12]:
# Pull sample for AIR_274
# keep = ['account', 'seq', 'id_powid', 'id_proxid', 'id_seleid', 'id_orgid', 'id_ultid', 'history_date', 'history_datetime','ver_src_id_list','empl_ct_best']
# base = base[keep]
# base[(base["ver_src_id_list"].str.contains("RR")) & (base["ver_src_id_list"].str.contains("Z1")) & (base.empl_ct_best ==0)].to_csv("./_temp/sample_274.csv", index = False)
# keep = ['account', 'seq', 'id_powid', 'id_proxid', 'id_seleid', 'id_orgid', 'id_ultid', 'history_date', 'history_datetime','ver_src_id_list','empl_ct_best']
# base = base[keep]
# # base[(~base["ver_src_id_list"].str.contains("RR")) & (~base["ver_src_id_list"].str.contains("Z1")) & (base.empl_ct_best >0)]
# base[(base.empl_ct_best >0) & (base.ver_src_id_list != "") & (~base.ver_src_id_list.str.contains("RR", na=False))  & (~base.ver_src_id_list.str.contains("Z1", na=False))].head(5).to_csv("./_temp/sample_274_with_ee_noZ1_noRR.csv", index = False)

In [8]:
exclude = ['account', 'seq', 'id_powid', 'id_proxid', 'id_seleid', 'id_orgid', 'id_ultid', 'id_seleid_change_flag', 'id_seleid_change_code', 'id_weight',
           'id_score', 'id_category', 'id_status', 'id_truebiz', 'history_date', 'history_datetime',]
sbfe_attr = [x for x in base.columns if "sbfe" in x]
rv_attr = [x for x in base.columns if x not in sbfe_attr and x not in exclude]

exception_dict = dict()
for x in base.columns:
    if x in sbfe_attr:
        exception_dict[x] = [-99, -98, -97]
    if x in rv_attr:
        exception_dict[x] = [-1]

In [9]:
from dsgtools.reporting import ImpactAnalysisReport
cr = ImpactAnalysisReport(base, test, keys="account",
           exceptions= exception_dict,
           dataframe_names = ("base", "test"))
cr.to_xlsx("./_temp/air_1587comparison.xlsx", overwrite= True)

In [4]:
base["input_value_le_total_value"] = np.where(base.addr_input_assessed_value <= base.prop_assessed_value_total, 1, 0)
test["input_value_le_total_value"] = np.where(test.addr_input_assessed_value <= test.prop_assessed_value_total, 1, 0)

In [13]:
# interested attributes: 'AssetPropAssessedTotalEver', "InputAddrOwnership", "InputAddrAssessedTotal"
# prop_assessed_value_total: -1, 0, pos
# addr_input_ownership: -1, 0, 1
# addr_input_assessed_value: -1, 0, pos
fmt = make_format(cuts = [-np.inf, 0, np.inf], exceptions = [-1])
freq("addr_input_ownership", "addr_input_assessed_value", "prop_assessed_value_total", "input_value_le_total_value", df = base, format = [None, fmt, fmt, None], cross = False, observed = True)

Count  \
addr_input_ownership addr_input_assessed_value prop_assessed_value_total input_value_le_total_value            
1                    <= 0                      <= 0                      1                              47.0   
                                               1+                        1                             244.0   
                     1+                        <= 0                      0                           14494.0   
                                               1+                        1                           19021.0   
                                                                         0                           65490.0   
0                    <= 0                      <= 0                      1                              15.0   
                                               1+                        1                              76.0   
                     1+                        <= 0                      0                             193.0   
                                               1+                        1                              69.0   
                                                                         0                             210.0   
-1                   1+                        -1                        0                               3.0   
                     -1                        -1                        1                             138.0   

                                                                                                         Pct  \
addr_input_ownership addr_input_assessed_value prop_assessed_value_total input_value_le_total_value            
1                    <= 0                      <= 0                      1                           0.00047   
                                               1+                        1                           0.00244   
                     1+                        <= 0                      0                           0.14494   
                                               1+                        1                           0.19021   
                                                                         0                           0.65490   
0                    <= 0                      <= 0                      1                           0.00015   
                                               1+                        1                           0.00076   
                     1+                        <= 0                      0                           0.00193   
                                               1+                        1                           0.00069   
                                                                         0                           0.00210   
-1                   1+                        -1                        0                           0.00003   
                     -1                        -1                        1                           0.00138   

                                                                                                     Cuml Count  \
addr_input_ownership addr_input_assessed_value prop_assessed_value_total input_value_le_total_value               
1                    <= 0                      <= 0                      1                                 47.0   
                                               1+                        1                                291.0   
                     1+                        <= 0                      0                              14785.0   
                                               1+                        1                              33806.0   
                                                                         0                              99296.0   
0                    <= 0                      <= 0                      1                              99311.0   
                                             

In [15]:
freq("addr_input_ownership", "addr_input_assessed_value", "prop_assessed_value_total", "input_value_le_total_value", df = test, format = [None, fmt, fmt, None], cross = False, observed = True)

Count  \
addr_input_ownership addr_input_assessed_value prop_assessed_value_total input_value_le_total_value            
1                    <= 0                      <= 0                      1                               1.0   
                                               1+                        1                             288.0   
                     1+                        <= 0                      0                             675.0   
                                               1+                        0                           65359.0   
                                                                         1                           32713.0   
0                    <= 0                      <= 0                      1                               1.0   
                                               1+                        1                              91.0   
                     1+                        <= 0                      0                             178.0   
                                               1+                        0                             185.0   
                                                                         1                              84.0   
-1                   1+                        -1                        0                               3.0   
                     -1                        -1                        1                             138.0   

                                                                                                          Pct  \
addr_input_ownership addr_input_assessed_value prop_assessed_value_total input_value_le_total_value             
1                    <= 0                      <= 0                      1                           0.000010   
                                               1+                        1                           0.002888   
                     1+                        <= 0                      0                           0.006769   
                                               1+                        0                           0.655451   
                                                                         1                           0.328062   
0                    <= 0                      <= 0                      1                           0.000010   
                                               1+                        1                           0.000913   
                     1+                        <= 0                      0                           0.001785   
                                               1+                        0                           0.001855   
                                                                         1                           0.000842   
-1                   1+                        -1                        0                           0.000030   
                     -1                        -1                        1                           0.001384   

                                                                                                     Cuml Count  \
addr_input_ownership addr_input_assessed_value prop_assessed_value_total input_value_le_total_value               
1                    <= 0                      <= 0                      1                                  1.0   
                                               1+                        1                                289.0   
                     1+                        <= 0                      0                                964.0   
                                               1+                        0                              66323.0   
                                                                         1                              99036.0   
0                    <= 0                      <= 0                      1                              99037.0   
                               

In [6]:
test.loc[(test.addr_input_ownership ==1) & (test.addr_input_assessed_value > 0) & (test.prop_assessed_value_total <=0) & (test.input_value_le_total_value ==0), 
         ["account", "id_seleid", 'prop_assessed_value_total', 'addr_input_ownership',"addr_input_assessed_value"]].head()

,account,id_seleid,prop_assessed_value_total,addr_input_ownership,addr_input_assessed_value
0,7450,450923292,144210,1,1442100
1,3709,93388275,127790,1,638950
2,5067,107586492,515000,1,587391
3,10106,19140904744,143680,1,359200
5,5322,134252400862,54500,1,278570


In [11]:
base.loc[(base.addr_input_ownership ==1) & (base.addr_input_assessed_value > 0) & (base.prop_assessed_value_total > 0) & (base.input_value_le_total_value ==0), 
         ["account", "id_seleid", 'prop_assessed_value_total', 'addr_input_ownership',"addr_input_assessed_value", "prop_count"]].head()

,account,id_seleid,prop_assessed_value_total,addr_input_ownership,addr_input_assessed_value,prop_count
1,802999,138709957624,287900,1,899700,1
2,802308,100952985,390200,1,433300,2
3,803525,41107903,5517,1,970320,1
4,803578,48068613,112960,1,753066,1
5,803393,27954612,224070,1,266020,1


In [10]:
base.loc[(base.id_seleid.isin([450923292, 93388275, 107586492, 19140904744, 134252400862])), 
         ["account", "id_seleid", 'prop_assessed_value_total', 'addr_input_ownership',"addr_input_assessed_value", "prop_count"]].head()

,account,id_seleid,prop_assessed_value_total,addr_input_ownership,addr_input_assessed_value,prop_count
21768,5067,107586492,0,1,587391,1
41238,5322,134252400862,0,1,278570,1
59105,3709,93388275,0,1,638950,1
75763,7450,450923292,0,1,1442100,1
75764,10106,19140904744,0,1,359200,1


In [5]:
test.loc[(test.addr_input_ownership ==1) & (test.addr_input_assessed_value > 0) & (test.prop_assessed_value_total > 0) & (test.input_value_le_total_value ==0), 
         ["account", "id_seleid", 'prop_assessed_value_total', 'addr_input_ownership',"addr_input_assessed_value", "prop_count"]].head()

,account,id_seleid,prop_assessed_value_total,addr_input_ownership,addr_input_assessed_value,prop_count
0,7450,450923292,144210,1,1442100,1
1,3709,93388275,127790,1,638950,1
2,5067,107586492,515000,1,587391,1
3,10106,19140904744,143680,1,359200,1
5,5322,134252400862,54500,1,278570,2


In [21]:
keep = ["account", "id_seleid", 'prop_assessed_value_total', 'prop_assessed_value_total_curr', 'prop_state_count_curr', 'prop_bldg_size_total_curr', 'prop_bldg_size_total', 
        'prop_count_curr', 'prop_lot_size_total', 'prop_count', 'prop_lot_size_total_curr', 'prop_state_count', 'addr_input_ownership', ]
test_temp = test[keep].copy()
test_temp.columns = ["t_" + x for x in test_temp.columns]
base_temp = base[keep].copy()

merged = test_temp.merge(base_temp, left_on = "t_account", right_on = "account")
print(merged[(merged.prop_assessed_value_total >0) & (merged.t_prop_assessed_value_total ==0)].shape)
merged[(merged.prop_assessed_value_total >0) & (merged.t_prop_assessed_value_total ==0)].head()

(26, 26)


,t_account,t_id_seleid,t_prop_assessed_value_total,t_prop_assessed_value_total_curr,t_prop_state_count_curr,t_prop_bldg_size_total_curr,t_prop_bldg_size_total,t_prop_count_curr,t_prop_lot_size_total,t_prop_count,...,prop_assessed_value_total_curr,prop_state_count_curr,prop_bldg_size_total_curr,prop_bldg_size_total,prop_count_curr,prop_lot_size_total,prop_count,prop_lot_size_total_curr,prop_state_count,addr_input_ownership
6757,1177087,44605384597,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,1,0,1,1
7321,1328497,33446559,0,0,1,0,0,1,1400,1,...,0,0,0,0,0,1400,1,0,1,1
11116,2163593,34576753,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
11728,2504154,4261857933,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
15357,445794,131810423,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,1,0,1,1


In [22]:
diff_list = ['prop_assessed_value_total', 'prop_assessed_value_total_curr', 'prop_state_count_curr', 'prop_bldg_size_total_curr', 'prop_bldg_size_total', 'prop_count_curr', 
             'prop_lot_size_total', 'prop_count', 'prop_lot_size_total_curr', 'prop_state_count', 'addr_input_ownership', ]

for x in diff_list:
    merged[x + "_diff"] = np.where((merged[x] < 0) & (merged["t_" + x] < 0), -999_999_999, np.where(
        (merged[x] >= 0) & (merged["t_" + x] < 0), -999_999_998, np.where(
            (merged[x] < 0) & (merged["t_" + x] >= 0), -999_999_997, merged["t_" + x] - merged[x]
        )
    ))

In [23]:
# pd.options.display.float_format = '{:.2f}'.format
# merged[[x + "_diff" for x in diff_list]].describe([0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95])
result = dict()
fmt_1 = make_format(cuts = [-np.inf, -500_000_001, -1_000_001, -500_001, -100_001, -50_001, -10_001, -5_001, -1, 0, 5_000, 10_000, 50_000, 
                            100_000, 500_000, 1_000_000, 500_000_000, np.inf], exceptions = [-999_999_999, -999_999_998, -999_999_997])
fmt_2 = make_format(cuts = [-np.inf, -51, -21, -16, -11, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 10, 15, 20, 50, np.inf], exceptions = [-999_999_999, -999_999_998, -999_999_997])
result["f1"] = freq(merged.addr_input_ownership_diff, observed = True)
result["f2"] = freq(merged.prop_assessed_value_total_diff, format = [fmt_1], observed = True)
result["f3"] = freq(merged.prop_assessed_value_total_curr_diff, format = [fmt_1], observed = True)
result["f4"] = freq(merged.prop_bldg_size_total_diff, format = [fmt_1], observed = True)
result["f5"] = freq(merged.prop_bldg_size_total_curr_diff, format = [fmt_1], observed = True)
result["f6"] = freq(merged.prop_lot_size_total_diff, format = [fmt_1], observed = True)
result["f7"] = freq(merged.prop_lot_size_total_curr_diff, format = [fmt_1], observed = True)
result["f8"] = freq(merged.prop_count_diff, format = [fmt_2], observed = True)
result["f9"] = freq(merged.prop_count_curr_diff, format = [fmt_2], observed = True)
result["f10"] = freq(merged.prop_state_count_diff, format = [fmt_2], observed = True)
result["f11"] = freq(merged.prop_state_count_curr_diff, format = [fmt_2], observed = True)
									
wb = TableWriter(filename = "./_temp/temp.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="fair_lending",
    )
wb.close()

In [26]:
merged[merged.addr_input_ownership_diff ==1].head()

,t_account,t_id_seleid,t_prop_assessed_value_total,t_prop_assessed_value_total_curr,t_prop_state_count_curr,t_prop_bldg_size_total_curr,t_prop_bldg_size_total,t_prop_count_curr,t_prop_lot_size_total,t_prop_count,...,prop_assessed_value_total_curr_diff,prop_state_count_curr_diff,prop_bldg_size_total_curr_diff,prop_bldg_size_total_diff,prop_count_curr_diff,prop_lot_size_total_diff,prop_count_diff,prop_lot_size_total_curr_diff,prop_state_count_diff,addr_input_ownership_diff
4454,2026561,98106493,135005,135005,1,6922,6922,1,18295,2,...,47715,1,4320,4320,0,18295,1,0,0,1
4463,2024680,28930161005,1089165,1089165,1,47780,47780,2,24592,2,...,0,1,0,0,1,24592,1,24592,1,1
4682,2087520,542701849,91631,0,0,0,3532,0,0,1,...,0,0,0,0,0,0,0,0,1,1
10165,2700636,105634882,9649000,9649000,1,12356,12356,2,0,3,...,8953400,0,11068,11068,1,0,2,0,0,1
10572,2876755,161397180,1486410,1486410,0,3770,3770,1,0,2,...,0,-1,0,0,0,0,1,0,0,1


In [28]:
merged[(merged.prop_count_diff ==-4)].head()

,t_account,t_id_seleid,t_prop_assessed_value_total,t_prop_assessed_value_total_curr,t_prop_state_count_curr,t_prop_bldg_size_total_curr,t_prop_bldg_size_total,t_prop_count_curr,t_prop_lot_size_total,t_prop_count,...,prop_assessed_value_total_curr_diff,prop_state_count_curr_diff,prop_bldg_size_total_curr_diff,prop_bldg_size_total_diff,prop_count_curr_diff,prop_lot_size_total_diff,prop_count_diff,prop_lot_size_total_curr_diff,prop_state_count_diff,addr_input_ownership_diff
40670,2165016,16077656,175620,126840,0,18240,23640,1,460,2,...,126840,0,18240,0,-4,0,-4,0,0,0
61027,3100841,58789806,845486,845486,1,6920,6920,1,4449,2,...,845486,0,6920,5321,0,-65682,-4,0,0,0


In [16]:
merged[(merged.prop_assessed_value_total_diff > -500000000) & (merged.prop_assessed_value_total_diff < -100000000)]

,t_account,t_id_seleid,t_prop_assessed_value_total,t_prop_assessed_value_total_curr,t_prop_state_count_curr,t_prop_bldg_size_total_curr,t_prop_bldg_size_total,t_prop_count_curr,t_prop_lot_size_total,t_prop_count,t_prop_lot_size_total_curr,t_prop_state_count,t_addr_input_ownership,account,id_seleid,prop_assessed_value_total,prop_assessed_value_total_curr,prop_state_count_curr,prop_bldg_size_total_curr,prop_bldg_size_total,prop_count_curr,prop_lot_size_total,prop_count,prop_lot_size_total_curr,prop_state_count,addr_input_ownership,prop_assessed_value_total_diff,prop_assessed_value_total_curr_diff,prop_state_count_curr_diff,prop_bldg_size_total_curr_diff,prop_bldg_size_total_diff,prop_count_curr_diff,prop_lot_size_total_diff,prop_count_diff,prop_lot_size_total_curr_diff,prop_state_count_diff,addr_input_ownership_diff
1940,1875285,13501654,383869,383869,1,580642,580642,1,0,1,0,1,1,1875285,13501654,241458830,241204630,1,580642,580642,1,0,1,0,1,1,-241074961,-240820761,0,0,0,0,0,0,0,0,0
17242,1300415,64622154,33274455,33267464,1,57323,1044237,2,0,2,0,1,1,1300415,64622154,149797544,106997544,1,313061,1299975,1,0,1,0,1,1,-116523089,-73730080,0,-255738,-255738,1,0,1,0,0,0
19776,2275539,150427651,886300,0,0,0,3615,0,160,3,0,1,1,2275539,150427651,171576408,171037008,1,0,3615,1,0,2,0,1,1,-170690108,-171037008,-1,0,0,-1,160,1,0,0,0
24526,1098992,63201768,16684670,16684670,1,407409,407409,1,0,1,0,1,1,1098992,63201768,175036100,175036100,1,407409,407409,1,0,1,0,1,1,-158351430,-158351430,0,0,0,0,0,0,0,0,0
31485,1260235,58574316,99007818,0,0,0,569304,0,0,1,0,0,0,1260235,58574316,237675674,237675674,0,107876,107876,1,0,1,0,1,1,-138667856,-237675674,0,-107876,461428,-1,0,0,0,-1,-1
40631,1509394,11447953,13690,0,0,0,125000,0,0,1,0,1,1,1509394,11447953,297592994,297592994,1,0,0,1,0,1,0,1,1,-297579304,-297592994,-1,0,125000,-1,0,0,0,0,0
41318,1723784,90839988,299100,0,1,0,273214,1,71516,1,71516,1,1,1723784,90839988,178035600,0,1,0,125726,1,71516,1,71516,1,1,-177736500,0,0,0,147488,0,0,0,0,0,0
53517,2668712,11262373,189583100,189583100,1,424316,424316,1,0,1,0,1,1,2668712,11262373,564873656,564873656,1,424316,424316,1,0,1,0,1,1,-375290556,-375290556,0,0,0,0,0,0,0,0,0
66973,3018075,53522960,123204000,123204000,1,0,0,1,0,1,0,1,1,3018075,53522960,238897400,238897400,1,492110,492110,1,0,1,0,1,1,-115693400,-115693400,0,-492110,-492110,0,0,0,0,0,0
67213,1432941,105643100,453317150,338859500,1,6097,409060,4,0,8,0,1,0,1432941,105643100,739375750,200000000,0,701,368035,1,0,5,0,2,1,-286058600,138859500,1,5396,41025,3,0,3,0,-1,-1


In [14]:
merged[(merged.addr_input_ownership ==1) & (merged.t_addr_input_ownership==0)]

,t_account,t_id_seleid,t_prop_assessed_value_total,t_prop_assessed_value_total_curr,t_prop_state_count_curr,t_prop_bldg_size_total_curr,t_prop_bldg_size_total,t_prop_count_curr,t_prop_lot_size_total,t_prop_count,t_prop_lot_size_total_curr,t_prop_state_count,t_addr_input_ownership,account,id_seleid,prop_assessed_value_total,prop_assessed_value_total_curr,prop_state_count_curr,prop_bldg_size_total_curr,prop_bldg_size_total,prop_count_curr,prop_lot_size_total,prop_count,prop_lot_size_total_curr,prop_state_count,addr_input_ownership,prop_assessed_value_total_diff,prop_assessed_value_total_curr_diff,prop_state_count_curr_diff,prop_bldg_size_total_curr_diff,prop_bldg_size_total_diff,prop_count_curr_diff,prop_lot_size_total_diff,prop_count_diff,prop_lot_size_total_curr_diff,prop_state_count_diff,addr_input_ownership_diff
108,920431,50204287,514899,514899,0,5000,5000,1,0,1,0,0,0,920431,50204287,60416,60416,1,5000,5000,1,0,1,0,1,1,454483,454483,-1,0,0,0,0,0,0,-1,-1
180,963606,96583435,0,0,0,0,0,0,0,0,0,0,0,963606,96583435,0,0,1,0,0,1,0,1,0,1,1,0,0,-1,0,0,-1,0,-1,0,-1,-1
195,1190833,122316337882,0,0,0,0,0,0,0,0,0,0,0,1190833,122316337882,0,0,1,0,0,1,100,1,100,1,1,0,0,-1,0,0,-1,-100,-1,-100,-1,-1
221,455436,60350772,171978,171978,0,11814,11814,1,0,1,0,0,0,455436,60350772,180656,180656,1,11814,11814,1,0,1,0,1,1,-8678,-8678,-1,0,0,0,0,0,0,-1,-1
351,1682514,65589880,280117,0,0,0,4400,0,0,1,0,0,0,1682514,65589880,152019,152019,1,0,0,1,0,1,0,1,1,128098,-152019,-1,0,4400,-1,0,0,0,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99828,22628,151672935,127511,0,0,0,954084,0,0,1,0,0,0,22628,151672935,3848516,3848516,1,431971,431971,1,0,1,0,1,1,-3721005,-3848516,-1,-431971,522113,-1,0,0,0,-1,-1
99843,2282994,101824669,479948,345661,0,288,3648,1,0,1,0,0,0,2282994,101824669,272202,272202,1,4852,4852,1,0,1,0,1,1,207746,73459,-1,-4564,-1204,0,0,0,0,-1,-1
99876,257310,69198389,1740,1740,0,0,0,1,0,1,0,0,0,257310,69198389,587640,1740,0,0,2385,1,0,1,0,1,1,-585900,0,0,0,-2385,0,0,0,0,-1,-1
99917,2911459,32943914,1366940,1366940,0,21100,21100,1,0,1,0,0,0,2911459,32943914,116689,116689,1,0,0,1,0,1,0,1,1,1250251,1250251,-1,21100,21100,0,0,0,0,-1,-1


In [19]:
pd.set_option('display.max_columns', None)
merged[(merged.prop_count ==1) & (merged.prop_assessed_value_total>0) & ((merged.t_prop_count >1) & (merged.t_prop_assessed_value_total>0))]

,t_account,t_id_seleid,t_prop_assessed_value_total,t_prop_assessed_value_total_curr,t_prop_state_count_curr,t_prop_bldg_size_total_curr,t_prop_bldg_size_total,t_prop_count_curr,t_prop_lot_size_total,t_prop_count,t_prop_lot_size_total_curr,t_prop_state_count,t_addr_input_ownership,account,id_seleid,prop_assessed_value_total,prop_assessed_value_total_curr,prop_state_count_curr,prop_bldg_size_total_curr,prop_bldg_size_total,prop_count_curr,prop_lot_size_total,prop_count,prop_lot_size_total_curr,prop_state_count,addr_input_ownership,prop_assessed_value_total_diff,prop_assessed_value_total_curr_diff,prop_state_count_curr_diff,prop_bldg_size_total_curr_diff,prop_bldg_size_total_diff,prop_count_curr_diff,prop_lot_size_total_diff,prop_count_diff,prop_lot_size_total_curr_diff,prop_state_count_diff,addr_input_ownership_diff
633,712566,136511808043,1051217,1051217,1,17592,17592,1,43410,2,43410,1,1,712566,136511808043,19970,19970,1,0,0,1,0,1,0,1,1,1031247,1031247,0,17592,17592,0,43410,1,43410,0,0
1531,1862699,78774055,53669,53669,1,891,891,2,0,2,0,1,1,1862699,78774055,53669,53669,1,891,891,1,0,1,0,1,1,0,0,0,0,0,1,0,1,0,0,0
2303,2855686,137402440065,75851,75851,0,5835,5835,1,0,2,0,1,1,2855686,137402440065,75851,75851,1,5835,5835,1,0,1,0,1,1,0,0,-1,0,0,0,0,1,0,0,0
2538,5987,160695779,661600,64900,1,2804,35684,2,0,2,0,1,1,5987,160695779,226350,0,0,0,30800,0,0,1,0,1,1,435250,64900,1,2804,4884,2,0,1,0,0,0
2556,5922,146108791,4233188,4233188,1,76177,76177,1,809,2,809,2,1,5922,146108791,247123,247123,0,11100,11100,1,809,1,0,1,1,3986065,3986065,1,65077,65077,0,0,1,809,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99685,841207,11103693,1376483,1376483,1,5730,5730,1,0,2,0,2,1,841207,11103693,1417233,1417233,1,4862,4862,1,0,1,0,1,1,-40750,-40750,0,868,868,0,0,1,0,1,0
99696,855969,71082998,823550,823550,0,16576,16576,1,0,2,0,1,1,855969,71082998,50930,0,0,0,0,0,0,1,0,1,1,772620,823550,0,16576,16576,1,0,1,0,0,0
99700,858474,26118893,908100,754800,1,10804,10804,1,0,2,0,1,1,858474,26118893,169200,0,0,0,0,0,0,1,0,1,1,738900,754800,1,10804,10804,1,0,1,0,0,0
99702,859485,6868389244,101080,101080,0,1528,1528,1,0,2,0,1,1,859485,6868389244,121589,121589,0,1528,1528,1,0,1,0,1,1,-20509,-20509,0,0,0,0,0,1,0,0,0


In [7]:
base_temp[(base_temp.prop_count >0) & (base_temp.prop_assessed_value_total<=0)]

,account,id_seleid,prop_assessed_value_total,prop_assessed_value_total_curr,prop_state_count_curr,prop_bldg_size_total_curr,prop_bldg_size_total,prop_count_curr,prop_lot_size_total,prop_count,prop_lot_size_total_curr,prop_state_count,addr_input_ownership
0,28547,28914536474,0,0,1,0,0,1,0,1,0,1,1
1,30314,727771974,0,0,0,0,0,0,0,1,0,1,1
2,20789,127476823,0,0,1,0,0,1,0,1,0,1,1
3,56728,161105404,0,0,0,0,0,0,0,1,0,1,1
4,38181,122659884,0,0,1,0,0,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99599,2907039,54237904276,0,0,0,0,0,0,0,1,0,1,1
99600,2908384,52958339027,0,0,0,0,0,0,0,1,0,1,1
99601,2911355,22435794292,0,0,0,0,0,0,0,1,0,1,1
99602,2919034,133787799955,0,0,1,0,0,1,0,1,0,1,1


In [8]:
merged.loc[merged.prop_count_compare <0, ["t_account", 't_id_seleid', "prop_count", "t_prop_count"]]

,t_account,t_id_seleid,prop_count,t_prop_count
22,1530356,19534803,1,2
73,552396,137393276,1,2
95,891650,1285986279,1,3
97,892468,18164533,3,4
125,968318,161110133,1,2
...,...,...,...,...
99961,941060,44591014075,2,5
99973,2134477,52005885,2,4
99977,2212131,21387926,2,3
99989,1412672,13419188,1,2


In [6]:
merged["prop_count_compare"] = merged.prop_count - merged.t_prop_count
freq(merged["prop_count_compare"])

,Count,Pct,Cuml Count,Cuml Pct
prop_count_compare,,,,
-32,1.0,0.000010,1.0,0.000010
-22,1.0,0.000010,2.0,0.000020
-14,1.0,0.000010,3.0,0.000030
-11,3.0,0.000030,6.0,0.000060
-9,1.0,0.000010,7.0,0.000070
-8,2.0,0.000020,9.0,0.000090
-7,4.0,0.000040,13.0,0.000130
-6,12.0,0.000120,25.0,0.000250
-5,29.0,0.000290,54.0,0.000540


In [13]:
test.prop_bldg_size_total.describe()

count    99997.00000
mean        -0.00098
std          0.03129
min         -1.00000
25%          0.00000
50%          0.00000
75%          0.00000
max          0.00000
Name: prop_bldg_size_total, dtype: float64

In [14]:
base.prop_bldg_size_total.describe()

count    9.999900e+04
mean     1.460349e+04
std      3.817340e+05
min     -1.000000e+00
25%      0.000000e+00
50%      2.640000e+03
75%      8.280000e+03
max      7.888696e+07
Name: prop_bldg_size_total, dtype: float64